# Introduction
This notebook was created to calculate the shortest path between nodes from the CDR generated OD for Bogota

In [1]:
import math
import pandas as pd
import networkx as nx
import osmnx as ox
import requests
import json 
from collections import Counter

# Import and format data
Here we use the ODs from our potential demand, i.e. the resultinf ODs after applying the rejection-sampling algorithm on the original ODs.

In [2]:
# These are ourODs after Rejection-Sampling method
nodes=pd.read_csv('nodes_all_trips.csv')
links=pd.read_csv('links_after_rejection.csv')

In [3]:
nodes.head(2)

,id,Label,income_level,wstrato,pop,Longitude,Latitude
0,1.0,1.0,2.0,3.000000,17197.0,-74.1205,4.61808
1,2.0,2.0,1.0,1.860132,24609.0,-74.1327,4.56084


In [4]:
links.head(2)

,Source,Target,Weight,income_level,wstrato_o,Type
0,265.0,455.0,28.228165,1.0,2.0,Directed
1,448.0,455.0,24.726087,2.0,3.0,Directed


In [5]:
# Add lat and long to the links origin and destination by merging table
merge_source=pd.merge(links,nodes[['id','Latitude','Longitude']],left_on='Source',right_on='id',how='left')

In [6]:
merge_source.rename(columns={'Latitude': 'source_lat', 'Longitude': 'source_lon'},inplace=True)

In [7]:
merge_target=pd.merge(merge_source,nodes[['id','Latitude','Longitude']],left_on='Target',right_on='id',how='left')

In [8]:
merge_target.rename(columns={'Latitude': 'target_lat', 'Longitude': 'target_lon'},inplace=True)

In [9]:
links=merge_target.drop(['id_x','id_y'],axis=1)

In [10]:
links.head(2)

,Source,Target,Weight,income_level,wstrato_o,Type,source_lat,source_lon,target_lat,target_lon
0,265.0,455.0,28.228165,1.0,2.0,Directed,4.73042,-74.1060,4.61034,-74.0704
1,448.0,455.0,24.726087,2.0,3.0,Directed,4.60370,-74.0725,4.61034,-74.0704


In [11]:
# Prepare the data for the query
links.loc[:,'coordinates_o_d']=links['source_lon'].astype('str')+','+links['source_lat'].astype('str')+';'+\
                               links['target_lon'].astype('str')+','+links['target_lat'].astype('str')

# Routing
Here we route the trips using Open Source Routing Machine (OSRM) project, an open{source efficient routing
software, and also configurable and extensible. For documentation, please see http://project-osrm.org/docs/v5.22.0/api/#stepmaneuver-object

Specifically we use OSRM-backend, thus we use Docker. Here two link for installation adn documentation purposes:
https://hub.docker.com/r/osrm/osrm-backend/
https://github.com/Project-OSRM/osrm-backend


In [18]:
def api_query(profile,coordinates):
    # url_base='http://router.project-osrm.org/route/v1/' 
    #Docker needs to be running before use this function
    url_base='http://localhost:5000/route/v1/'
    url=url_base+profile+'/'+coordinates+'?steps=false&geometries=polyline&overview=full&annotations=true'
    # Make the query
    response = requests.get(url)
    result=response.text
    dict_result=json.loads(result)
    return dict_result

In [19]:
def read_result(result):
    nodes=[]
    legs=result['routes'][0]['legs']
    for leg in range(len(legs)):
        nodes.append(legs[leg]['annotation']['nodes'])
        flat_list=[y for x in nodes for y in x]
    return flat_list

In [28]:
#Set the profile (cars,bicycle of walk) or your own profile, you can modified the .lua files of the previous mentioned profile
profile='bicycle'
for index,row in links.iterrows():
    result=api_query(profile,row['coordinates_o_d'])
    list_nodes=read_result(result)
    # Need to fix the object part
    links=links.astype('object')
    links.loc[index,'nodes']=list_nodes
    links.loc[index,'distance']=result['routes'][0]['distance']

In [32]:
links.to_csv('latent_demand_p_bikes.csv')

# Building the flow network

In [2]:
#load the routes for all OD pairs
links=pd.read_csv('latent_demand_p_bikes.csv')

In [3]:
links.loc[:,'node_seq']=links['nodes'].apply(lambda x: list(x[1:-1].split(', ')))

In [4]:
# Function to calculate distance
def calculate_dist (lat_1,lat_2,lon_1,lon_2):
    if (math.isnan(lat_1)|math.isnan(lat_2)|math.isnan(lon_1)|math.isnan(lon_2)):
         return float('nan')         
    elif ((lat_1==lat_2)&(lon_1==lon_2)):
        dist=0.0
    else:
        
        R_aver = 6374;
        lat_1 = math.radians(lat_1)
        lon_1 = math.radians(lon_1)
        lat_2 = math.radians(lat_2)
        lon_2 = math.radians(lon_2)
        dist = R_aver * math.acos(math.cos(lat_1)*math.cos(lat_2)*math.cos(lon_1-lon_2) + math.sin(lat_1)*math.sin(lat_2));
    return dist*1000

In [5]:
links.loc[:,'euclidean_dist']=links.apply(lambda row: calculate_dist(row['source_lat'],row['target_lat'],\
                                row['source_lon'],row['target_lon']),axis=1)/1000

In [6]:
# Create the pairs, nodes from every link
links.loc[:,'pairs']=links['node_seq'].apply(lambda x: list(zip(x[0:-1],x[1:])))

In [7]:
# Filter the data
links=links[(links['distance']<=10000)&(links['Weight']>=25)]

In [8]:
list_list=links['pairs'].tolist()
flat_list=[y for x in list_list for y in x]

In [9]:
#build the network
g = nx.DiGraph((x, y, {'weight': v}) for (x, y), v in Counter(flat_list).items())

In [10]:
pairs=pd.DataFrame([[x, y, v] for (x, y), v in Counter(flat_list).items()])
pairs.columns=['source','target','weight']

In [11]:
pairs.set_index('source',inplace=True)

In [12]:
pairs.shape

(93781, 2)

In [13]:
pairs.head(2)

,target,weight
source,,
267021465,5750230778,21
5750230778,267021464,21


In [14]:
pairs['weight'].describe()

count    93781.000000
mean        12.306032
std         15.350052
min          1.000000
25%          3.000000
50%          7.000000
75%         16.000000
max        197.000000
Name: weight, dtype: float64

### Get the nodes coordinates from OSM

In [17]:
# Get lat and lon of all nodes to set the bbox
y=links['source_lat'].tolist()
for elem in links['target_lat'].tolist():
    y.append(elem)
x=links['source_lon'].tolist()
for elem in links['target_lon'].tolist():
    x.append(elem)    

In [18]:
west=min(x)-0.01
east=max(x)+0.01
north=max(y)+0.01
south=min(y)-0.01
west,east,north,south

(-74.217, -74.0144, 4.82472, 4.48311)

In [19]:
# create network from that bounding box
# all_private - download all OSM streets and paths, including private-access ones (from documentation)
G1 = ox.graph_from_bbox(north, south, east, west, network_type='all_private',simplify=False)

In [54]:
nodes_bog=pd.DataFrame(index=[],columns=['node_id'])
nodes_bog.loc[:,'node_id']=list(G1)

In [55]:
nodes_bog.loc[:,'lat']=nodes_bog['node_id'].apply(lambda x: G1.node[x]['y'])
nodes_bog.loc[:,'lon']=nodes_bog['node_id'].apply(lambda x: G1.node[x]['x'])

In [94]:
# Nodes bog have the nodes id and coordinate for a certain bbox
nodes_bog.head(2)

,node_id,lat,lon
0,4687304058,4.555589,-74.097681
1,4687304059,4.555606,-74.097673


In [95]:
nodes_bog.rename(columns={'node_id':'id'}).set_index('id').to_csv('nodes_latent_bike.csv')

Get node lat and lon for the pairs

In [58]:
pairs.reset_index(inplace=True)

In [59]:
# Nodes from the pairs
all_nodes=pairs['source'].tolist()
for elem in pairs['target'].tolist():
    all_nodes.append(elem)

In [60]:
unique_nodes=list(set(all_nodes))

In [61]:
series=pd.Series(unique_nodes)
df_nodes=series.reset_index()
df_nodes.rename(columns={0:'node_id'},inplace=True)
df_nodes.loc[:,'node_id']=df_nodes['node_id'].astype('str')

In [62]:
# make the ids the same data type for the merge
nodes_bog.loc[:,'node_id']=nodes_bog['node_id'].astype('str')

In [63]:
merged=pd.merge(df_nodes,nodes_bog,left_on='node_id',right_on='node_id',how='left')

In [64]:
# Check wether a node was left outside
merged[merged['lat'].isnull()].shape

(82, 4)

### Get the nodes that do not have lat and lon

In [101]:
def api(node):
    response = requests.get('http://api.openstreetmap.org/api/0.6/node/'+str(node))
    result=response.text
    return result    

In [118]:
merged[merged['lat'].isnull()]

,index,node_id,lat,lon
50941,50941,5220256902,NaN,NaN


In [119]:
api(merged.loc[50941,['node_id']])

'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>404 Not Found</title>\n</head><body>\n<h1>Not Found</h1>\n<p>The requested URL /api/0.6/node/node_id    5220256902\nName: 50941, dtype: object was not found on this server.</p>\n<hr>\n<address>Apache/2.4.29 (Ubuntu) Server at api.openstreetmap.org Port 80</address>\n</body></html>\n'

### Network edges

In [20]:
lista=[]
for line in nx.generate_edgelist(G1, data=False):
    lista.append(line)

In [21]:
df=pd.DataFrame(index=[list(range(len(lista)))],columns=['lista','source','target'])

In [22]:
df.loc[:,'lista']=lista

In [26]:
df.head()

,lista,source,target
0,262391492 5728971089,262391492,5728971089
1,2272948048 2272948045,2272948048,2272948045
2,2272948048 2272948047,2272948048,2272948047
3,494927893 494927791,494927893,494927791
4,5098176538 5098176539,5098176538,5098176539


In [24]:
df.loc[:,'source']=df['lista'].apply(lambda x: x.split()[0])

In [25]:
df.loc[:,'target']=df['lista'].apply(lambda x: x.split()[1])

In [27]:
df.loc[:,'pairs']=df.apply(lambda row: ((row['source']),(row['target'])),axis=1)

In [28]:
df.loc[:,'pairs_2']=df.apply(lambda row: ((row['target']),(row['source'])),axis=1)

In [29]:
df.head(2)

,lista,source,target,pairs,pairs_2
0,262391492 5728971089,262391492,5728971089,"(262391492, 5728971089)","(5728971089, 262391492)"
1,2272948048 2272948045,2272948048,2272948045,"(2272948048, 2272948045)","(2272948045, 2272948048)"


In [32]:
pairs.reset_index(inplace=True)

In [33]:
pairs.loc[:,'source']=pairs['source'].astype('str')
pairs.loc[:,'target']=pairs['target'].astype('str')

In [34]:
pairs.loc[:,'pairs']=pairs.apply(lambda row: ((row['source']),(row['target'])),axis=1)

In [35]:
merge=pd.merge(df,pairs[['pairs','weight']],left_on='pairs',right_on='pairs',how='left')

In [36]:
pairs_notfound=pairs[~pairs['pairs'].isin(merge[merge['weight'].notnull()]['pairs'].tolist())]

In [37]:
# check how many pairs do we have left that could not be matched
pairs_notfound.shape,pairs.shape

((2156, 4), (93781, 4))

In [38]:
merge.drop(['lista'],axis=1,inplace=True)

In [39]:
merge.head(2)

,source,target,pairs,pairs_2,weight
0,262391492,5728971089,"(262391492, 5728971089)","(5728971089, 262391492)",NaN
1,2272948048,2272948045,"(2272948048, 2272948045)","(2272948045, 2272948048)",NaN


In [40]:
swap_od=pd.merge(merge,pairs_notfound[['pairs','weight']],left_on='pairs_2',right_on='pairs',how='left')

In [44]:
swap_od['weight_x'].fillna(0,inplace=True)
swap_od['weight_y'].fillna(0,inplace=True)
swap_od['weight_raw']=swap_od['weight_x']+swap_od['weight_y']

In [45]:
# This is the amount of links we have, from now on, we are going to add one weight to be able to draw in gephi
swap_od['weight_raw'].sum()

1147346.0

In [65]:
# check we had assigned all the links to the network or the not_found group
pairs['weight'].sum()-pairs_notfound2['weight'].sum()

1147346

In [46]:
swap_od['weight']=swap_od['weight_raw']+1
swap_od['weight'].sum()

1637201.0

In [60]:
pairs_notfound2=pairs[(~pairs['pairs'].isin(df['pairs'].tolist()))&(~pairs['pairs'].isin(df['pairs_2'].tolist()))]
pairs_notfound2.shape

(380, 4)

In [57]:
# this is the resulting Flow Network
swap_od[['source','target','weight']].set_index('source').to_csv('pairs_latent_bike_20190401.csv')

## Checking errors and duplicates

In [71]:
# df.loc[:,'weight']=0

In [72]:
# df.drop('lista',axis=1,inplace=True)

In [73]:
# pairs.loc[:,'source']=pairs['source'].astype('str')
# pairs.loc[:,'target']=pairs['target'].astype('str')

In [58]:
# df.head(2)

In [59]:
# df.shape[0]-df.drop_duplicates(subset=['source','target']).shape[0]

In [76]:
# Drop duplicates cos they are an error
# df.drop_duplicates(subset=['source','target'],inplace=True)

In [48]:
# pairs.head(2)

In [49]:
# # Check we do not have duplicates, which would mean we did something wrong
# pairs.shape[0]-pairs.drop_duplicates(subset=['source','target']).shape[0]

In [79]:
# for index,row in pairs.iterrows():
#     if not df[(df['source']==row['source'])&(df['target']==row['target'])].empty:
#         df.loc[(df['source']==row['source'])&(df['target']==row['target']),'weight']=row['weight']
#     else:
#         if not df[(df['target']==row['source'])&(df['source']==row['target'])].empty:
#             df.loc[(df['target']==row['source'])&(df['source']==row['target']),'weight']=row['weight']
#         else:
#             pairs.loc[index,'error']=1

In [50]:
# df['weight'].sum(),pairs['weight'].sum()

In [51]:
# pairs[pairs['error']==1].shape

In [52]:
# df_back_up=df

In [53]:
# df.loc[:,'weight']=df['weight']+1

In [54]:
# df.set_index('source').to_csv('pairs_latent_bike.csv')